In [1]:
from google.colab import files
uploaded = files.upload()

Saving Bakkhali_7_years_weather_data.csv to Bakkhali_7_years_weather_data.csv


In [3]:
import pandas as pd
df = pd.read_csv("Bakkhali_7_years_weather_data.csv")
df.head()

,Unnamed: 0,Datetime,Wind Speed (m/s),Temperature(°C),Radiation(W/m^2),Cloud_Coverage(%),Rain_(mm/hour),Relative_Humidity(%)
0,0,2019-01-01 00:00:00,4.26,17.56,0.0,1.39,0.0,71.43
1,1,2019-01-01 01:00:00,4.40,17.11,0.0,2.08,0.0,73.89
2,2,2019-01-01 02:00:00,4.67,16.61,0.0,2.78,0.0,76.95
3,3,2019-01-01 03:00:00,5.05,16.10,0.0,3.47,0.0,79.93
4,4,2019-01-01 04:00:00,5.31,15.58,0.0,6.27,0.0,82.74


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

In [5]:
df["Datetime"] = pd.to_datetime(df["Datetime"])

In [6]:
df = df.sort_values("Datetime").reset_index(drop=True)

In [7]:
df["hour"] = df["Datetime"].dt.hour
df["dayofweek"] = df["Datetime"].dt.dayofweek
df["month"] = df["Datetime"].dt.month
df["dayofyear"] = df["Datetime"].dt.dayofyear


In [8]:
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)


In [9]:
feature_cols = [
    "hour_sin", "hour_cos",
    "month_sin", "month_cos",
    "dayofweek", "dayofyear"
]

target_cols = [
    "Temperature(°C)",
    "Radiation(W/m^2)",
    "Cloud_Coverage(%)",
    "Rain_(mm/hour)",
    "Relative_Humidity(%)"
]

In [14]:
df = df.dropna().reset_index(drop=True)

In [15]:
X = df[feature_cols]
Y = df[target_cols]

In [16]:
train_size = int(len(df) * 0.8)

X_train = X.iloc[:train_size]
X_test  = X.iloc[train_size:]

Y_train = Y.iloc[:train_size]
Y_test  = Y.iloc[train_size:]

In [17]:
base_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model = MultiOutputRegressor(base_model)

model.fit(X_train, Y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=300, n_jobs=None,
                                            num_parallel_tree=None, ...))

In [18]:
predictions = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(Y_test, predictions))
r2 = r2_score(Y_test, predictions)

print("Overall RMSE:", rmse)
print("Overall R2 Score:", r2)

Overall RMSE: 42.73925360635261
Overall R2 Score: 0.5264030694961548


In [19]:
future_datetime = pd.DataFrame({
    "Datetime": ["2026-03-20 14:00:00"]
})

future_datetime["Datetime"] = pd.to_datetime(future_datetime["Datetime"])

In [20]:
future_datetime["hour"] = future_datetime["Datetime"].dt.hour
future_datetime["dayofweek"] = future_datetime["Datetime"].dt.dayofweek
future_datetime["month"] = future_datetime["Datetime"].dt.month
future_datetime["dayofyear"] = future_datetime["Datetime"].dt.dayofyear

future_datetime["hour_sin"] = np.sin(2 * np.pi * future_datetime["hour"] / 24)
future_datetime["hour_cos"] = np.cos(2 * np.pi * future_datetime["hour"] / 24)

future_datetime["month_sin"] = np.sin(2 * np.pi * future_datetime["month"] / 12)
future_datetime["month_cos"] = np.cos(2 * np.pi * future_datetime["month"] / 12)

future_X = future_datetime[feature_cols]

future_prediction = model.predict(future_X)

print("\nPrediction for future datetime:")
print("Temperature:", future_prediction[0][0])
print("Radiation:", future_prediction[0][1])
print("Cloud:", future_prediction[0][2])
print("Rain:", future_prediction[0][3])
print("Humidity:", future_prediction[0][4])


Prediction for future datetime:
Temperature: 28.942543
Radiation: 570.6082
Cloud: 51.492584
Rain: 2.372955
Humidity: 60.162712


In [22]:
wind_target = df["Wind Speed (m/s)"]


In [23]:
Y_train_wind = wind_target.iloc[:train_size]
Y_test_wind  = wind_target.iloc[train_size:]


In [24]:
wind_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

wind_model.fit(X_train, Y_train_wind)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [25]:
wind_predictions = wind_model.predict(X_test)

wind_rmse = np.sqrt(mean_squared_error(Y_test_wind, wind_predictions))
wind_r2 = r2_score(Y_test_wind, wind_predictions)

print("Wind RMSE:", wind_rmse)
print("Wind R2:", wind_r2)


Wind RMSE: 2.1700476023807296
Wind R2: 0.10890817658224339


In [26]:
future_wind_prediction = wind_model.predict(future_X)

print("Wind Speed:", future_wind_prediction[0])


Wind Speed: 3.5535276
